In [1]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
#
# Author: Yusuke Yamamoto
# Email: yusuke@hontolab.org
# URL: https://hontolab.org/
# Licence: MIT License

''' BERTを使った文のベクトル化．
    東北大学の乾研究室が構築したBERTモデルを使用．
    MeCabを用いて構築されたモデルであること，公式のtransformersにデフォルトで
    組み込まれていることが採用の理由．
'''
import numpy as np
import torch
from transformers import BertJapaneseTokenizer, BertModel

BERT_MODEL_PATH = "cl-tohoku/bert-base-japanese-whole-word-masking"


class BertVectorizer():
    ''' BERTを用いて文をベクトル化するクラス
    '''
    def __init__(self, bert_path=BERT_MODEL_PATH):
        self.bert = BertModel.from_pretrained(bert_path)
        self.tokenizer = BertJapaneseTokenizer.from_pretrained(bert_path)


    def vectorize(self, texts, pooling_strategy="REDUCE_MEAN"):
        # わかちがき
        encoded_data = self.tokenizer.batch_encode_plus(
            texts, pad_to_max_length=True, add_special_tokens=True)

        # BERT適用
        outputs = self.bert(torch.tensor(encoded_data['input_ids']))

        # BERTの隠れ層の最終レイヤーを取得
        last_hidden_layers = outputs[0]

        if pooling_strategy == "REDUCE_MEAN":
            return torch.mean(last_hidden_layers, dim=1).detach().numpy()
        elif pooling_strategy == "REDUCE_MAX":
            vector, _ = torch.max(last_hidden_layers, dim=1)
            return vector.detach().numpy()
        elif pooling_strategy == "REDUCE_MEAN_MAX":
            max_vector, _ = torch.max(last_hidden_layers, dim=1)
            mean_vector = torch.mean(last_hidden_layers, dim=1)
            return torch.cat((max_vector, mean_vector), dim=1).detach().numpy()
        elif pooling_strategy == "CLS_TOKEN":
            return last_hidden_layers[:, 0, :].detach().numpy()
        else:
            raise ValueError("specify valid pooling_strategy: {REDUCE_MEAN, REDUCE_MAX, REDUCE_MEAN_MAX, CLS_TOKEN}")

            
def cosine_similarity(v1, v2):
    norm1 = np.linalg.norm(v1)
    norm2 = np.linalg.norm(v2)
    inner_product = np.dot(v1, v2)
    return inner_product / (norm1 * norm2)


if __name__ == '__main__':
    texts = ["夜の街にオープンしているワイナリーの入り口",
             "踏切の近くにワイナリーが開店している",
             "野球選手のフォームの確認をしている"]

    vectorizer = BertVectorizer()

    # textsに格納された複数の文をバッチでベクトル化
    vectors = vectorizer.vectorize(texts, "CLS_TOKEN")

    # コサイン類似度計算（文0と文2のベクトルの類似度）
    cosine_similarity(vectors[0], vectors[2])

/Users/wyuki/opt/anaconda3/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:1767: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [2]:
import pandas as pd
from sklearn.decomposition import PCA
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
import itertools
from scipy.spatial import distance

In [3]:
#作成したアイデアのデータ
with open('/Users/wyuki/研究/ideas.csv') as f:
    test_data = f.readlines()
idea_texts = []
for line in test_data:
    idea_texts.append(line.replace('\ufeff', '').replace('\n',''))
print(idea_texts)

['ガンダムみたいな乗り物', '美容院', '登り棒', 'ソーシャルディスタンスを保てる', '乗った人の犯罪係数がわかる', 'Wi-Fiと充電機能付き', '新しいエレベーター', '図書館', '動くドライブスルー', '家から遠隔操作できる車', 'いるべき車線がARで表示される', '動く家', '方向音痴でも運転できる', '着る車', 'ドラえもんみたいに３mm浮いている', 'ベッド', '18歳以下でも運転できる', '外の景色を画像認識して何か教えてくれる', 'イス', 'タイヤが球体', '水陸空両用車', '遊牧民', '乗れば空間充電される', '無人デリバリー専用車', '気まずくならない車', 'いいと思った窓の景色を撮ってくれる', 'ブレスト車', 'タイムマシン', '動く道路', '道路がポイント切り替え', '優しい気持ちになれる車', '街中の建物にタイヤをつける', '自転車についてくる荷物カゴ', '車っぽい買い物袋', '消臭車', '絶対にドライブデートが成功する', 'しりぞけあう車', '道路でスキー', 'ドローンで運ばれる', '人力', 'UFOキャッチャー', 'どこでもドア', 'カラオケ', '渋滞の解消', '光の速さで動く', '街中ジェットコースター', 'フロントにいろいろ表示', 'くぐれる', 'またげる', '馬', 'カスタマイズできる車', '外側が液晶画面', '飛んでいるように感じる車', '悪路をなんとかしてくれる', 'ついてくる影', 'ついてくる傘', 'どこでも止まれる電車', '折り畳める', '外の広告とAmazonが連動', '着せ替えできる', '時間ぴったりの動画が流れる', '助手席がDJ', '夜は荷物うを勝手に運ぶ車', '椅子が増える', 'エンジンからオーダーメイドの車', '野生のタクシー', '乗った車が自分のものになる', 'ぽいぽいカプセル', 'たくさんの車でグリッドコンピューティング', '窓ガラスが液晶ディスプレイ', '実際より速く感じる車', 'トイレ', '窓に電子広告', '乗ってる人全員で運転', '道路を舗装する車', '道路を掃除する車', '酔わない車', 'パーソナルモビリティが合体', '目的地がランダムなタク

In [9]:
idea_texts2 = ['バチェラー', 'テレビ', '人', 'パーティ', '薔薇', 'お見合い', 'オファー']

In [5]:
#アイデアデータをBERTでベクトル化 １アイデアづつ
vector = vectorizer.vectorize([idea_texts[0]], "REDUCE_MEAN")
ideas_df = pd.DataFrame(vector, index=[idea_texts[0]])

for idea in idea_texts[1:]:
    vector = vectorizer.vectorize([idea], "REDUCE_MEAN")
    idea_df = pd.DataFrame(vector, index=[idea])
    ideas_df = pd.concat([ideas_df, idea_df])

ideas_df

,0,1,2,3,4,5,6,7,8,9,...,758,759,760,761,762,763,764,765,766,767
ガンダムみたいな乗り物,0.274816,0.009747,-0.083723,0.012144,-0.171453,0.176695,0.061611,0.075911,-0.143778,0.072328,...,-0.456604,-0.199730,0.031235,0.377395,-0.130521,-0.060932,0.310353,-0.082851,0.253293,-0.184184
美容院,-0.025080,-0.603679,-0.212934,-0.226308,-0.210158,0.055734,-0.017595,-0.134783,-0.141619,-0.115425,...,-0.018006,-0.432034,-0.612187,0.394253,0.020946,0.278899,-0.090007,-0.330896,-0.461628,-0.166597
登り棒,-0.668403,-0.360016,-0.480848,-0.341426,-0.743916,0.054159,-0.286817,0.090023,-0.101817,-0.112155,...,-0.137437,0.088582,-0.180156,0.120384,-0.089087,-0.269293,-0.115672,-0.257663,0.000215,-0.468838
ソーシャルディスタンスを保てる,0.370291,0.191143,-0.413574,0.250565,-0.276299,0.221515,-0.078784,0.137053,0.306942,-0.119385,...,-0.329675,-0.154053,0.020058,0.085920,0.183299,-0.226643,0.071383,-0.151370,-0.076495,0.039454
乗った人の犯罪係数がわかる,0.004799,-0.046344,-0.154747,-0.330349,-0.315760,0.427067,-0.402360,-0.001927,0.180523,0.210186,...,-0.266514,0.144489,0.081410,0.294400,-0.119593,-0.502034,-0.010618,-0.559290,0.131491,0.048123
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
VR＋BOXの旅行体験トラック,-0.475347,-0.098465,-0.434752,-0.630011,-0.679625,-0.259324,-0.221436,-0.311440,-0.198933,0.016400,...,-0.026777,0.036009,0.291430,0.305086,-0.262702,0.303221,-0.294431,-0.224006,0.060268,0.312001
階段を登るのが楽になる靴や鞄,-0.084250,-0.212842,-0.252523,-0.031667,-0.526179,0.058275,-0.029784,0.077513,-0.230904,-0.024937,...,-0.070462,-0.100560,0.198672,0.207354,-0.227350,0.100201,-0.091952,-0.667860,0.016228,-0.009499
回転寿司が家を通る,-0.116068,-0.119867,0.194889,0.045114,-0.285720,-0.256854,-0.054369,0.042614,0.180963,-0.198160,...,-0.052944,-0.142027,-0.213388,0.134858,-0.510963,-0.148498,0.083674,-0.078541,0.277254,-0.127468
移動式スーパーマーケット,0.034785,-0.143671,-0.208227,-0.078935,-0.083181,-0.193970,0.052409,0.001750,-0.179369,0.308509,...,-0.038094,-0.213763,-0.401538,0.356014,-0.278820,0.026956,-0.064660,-0.212547,-0.339331,-0.078683


In [11]:
#アイデアデータをBERTでベクトル化 １アイデアづつ
vector = vectorizer.vectorize([idea_texts2[0]], "REDUCE_MEAN")
ideas_df3 = pd.DataFrame(vector, index=[idea_texts2[0]])

for idea in idea_texts2[1:]:
    vector = vectorizer.vectorize([idea], "REDUCE_MEAN")
    idea_df = pd.DataFrame(vector, index=[idea])
    ideas_df3 = pd.concat([ideas_df3, idea_df])

ideas_df3

,0,1,2,3,4,5,6,7,8,9,...,758,759,760,761,762,763,764,765,766,767
バチェラー,-0.010535,0.250081,-0.250064,0.261034,-0.225966,0.450759,-0.266530,0.009788,0.352739,-0.042531,...,0.301409,0.328661,-0.281012,-0.263652,-0.329341,-0.456182,-0.167032,0.017876,0.451373,-0.487371
テレビ,0.060370,-0.371359,-0.462199,0.093036,-0.188733,-0.185142,-0.134208,-0.132561,-0.085853,0.315603,...,0.135737,0.381614,-0.395731,0.405795,-0.141851,-0.082604,0.395898,-0.052485,-0.738310,0.178817
人,-0.183977,0.528512,-0.551319,-0.422774,0.080082,0.581268,-0.053678,-0.098613,0.033792,0.022314,...,-0.196414,-0.205190,-0.865048,0.701440,-0.532941,0.145926,0.131339,-0.260669,-0.301465,-0.393588
パーティ,-0.617754,-0.363672,0.268564,-0.213456,0.070972,0.499222,0.274427,0.407901,-0.332722,-0.069954,...,-0.168135,-0.688271,-0.304964,0.769086,-0.040985,-0.576927,-0.717911,-0.169657,-1.144008,0.213485
薔薇,-0.377872,0.053957,-0.742262,-0.364813,-0.047049,-0.055750,0.052580,0.038865,0.032476,-0.005832,...,-0.015955,-0.240081,-0.437918,0.307578,-0.024691,-0.006212,0.161773,-0.264183,-0.063017,0.087889
お見合い,-0.333739,-0.609849,0.311443,-0.063497,-0.326058,0.202758,-0.308041,-0.544093,0.276087,-0.250562,...,-0.120942,-0.091727,-0.446846,-0.031696,-0.256337,-0.488042,-0.160468,0.145030,-0.432015,-0.034542
オファー,-0.256128,0.265397,-0.138818,-0.396268,-0.206473,0.764353,-0.148540,0.050296,0.149390,-0.284476,...,-0.176015,-0.485519,0.014179,0.261626,-0.058822,-0.182866,0.078274,-0.144829,-0.437109,0.114246


In [1]:
#アイデアデータをBERTでベクトル化 １アイデアづつ
vector = vectorizer.vectorize([idea_texts[0]], "REDUCE_MEAN")
ideas_df2 = pd.DataFrame(vector, index=[idea_texts[0]])

for idea in idea_texts[1:]:
    vector = vectorizer.vectorize([idea], "REDUCE_MEAN")
    idea_df = pd.DataFrame(vector, index=[idea])
    ideas_df2 = pd.concat([ideas_df2, idea_df])

ideas_df2

NameError: name 'vectorizer' is not defined

In [33]:
#新しい単語，アイデア
new_idea_texts = ['タケコプター']
new_vectors = vectorizer.vectorize(new_idea_texts, "CLS_TOKEN")
new_idea_df = pd.DataFrame(new_vectors, index=new_idea_texts)
new_idea_df.head(1)

,0,1,2,3,4,5,6,7,8,9,...,758,759,760,761,762,763,764,765,766,767
タケコプター,-0.147127,0.124564,-0.809067,-0.169035,-0.288068,0.860786,-0.128522,-0.212217,0.015344,0.512473,...,0.258177,-0.23562,-0.54919,-0.093486,-0.276989,-0.055753,0.478265,-0.432092,-0.416936,0.078901


## BERTでベクトル化　kmeansでクラスタリング　第二主成分が最も効いているクラスタの判定　新しいアイデアがそのクラスタに分類されるか？

In [146]:
def which_cluster_new_idea(ideas_df, new_idea_text):
    
    component_num = 3
    cluster_num = 10
    pca = PCA(n_components=component_num) #インスタンス生成
    ideas_matrix_pca = pca.fit_transform(ideas_df) #各主成分の値をベクトル化して返す　
    ideas_df_pca = pd.DataFrame(ideas_matrix_pca)# データフレームに格納
    ideas_df_pca.columns = ["PC{}".format(i+1) for i in range(component_num)]#name colums:PC1~2

    #k_meansでクラスタリング　第二主成分でクラスタリング
    kmeans = KMeans(n_clusters=cluster_num, random_state=0).fit(ideas_df_pca.loc[:, ['PC2', 'PC3']])
    ideas_df_pca['cluster_id'] = kmeans.labels_


    #新しいアイデアがどのクラスターに分類されるか？
    new_vectors = vectorizer.vectorize([new_idea_text], "REDUCE_MEAN") # 
    new_idea_vec_df = pd.DataFrame(new_vectors, index=[new_idea_text])
    new_idea_pca = pca.transform(new_idea_vec_df)
    new_idea_pca_df = pd.DataFrame(new_idea_pca)
    new_idea_pca_df.columns = ["PC{}".format(i+1) for i in range(component_num)]#name colums:PC1~
    cluster_new_idea = kmeans.predict(new_idea_pca_df.loc[:, ['PC2', 'PC3']])[0]
    
    #どのクラスターに何が含まれているかの確認
    cluster_dic = {}
    for cluster_id in range(cluster_num):
        cluster_contents = []
        for row_id in range(len(ideas_df_pca.values)):
            idea_cluster = ideas_df_pca.iat[row_id, 3]
            if cluster_id == idea_cluster:
                cluster_contents.append(idea_texts[row_id])
        cluster_dic[cluster_id] = cluster_contents
    
    return cluster_new_idea, cluster_dic[cluster_new_idea]

In [166]:
which_cluster_new_idea(ideas_df2, 'ホワイトベース')

/Users/wyuki/opt/anaconda3/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:1767: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


(0,
 ['登り棒',
  '乗った人の犯罪係数がわかる',
  'ドラえもんみたいに３mm浮いている',
  '18歳以下でも運転できる',
  '外の景色を画像認識して何か教えてくれる',
  'タイムマシン',
  'カラオケ',
  '光の速さで動く',
  '折り畳める',
  '時間ぴったりの動画が流れる',
  '椅子が増える',
  'エンジンからオーダーメイドの車',
  '乗った車が自分のものになる',
  '実際より速く感じる車',
  'トイレ',
  'お父さんお母さんと連携してサンタさん',
  '出口までお待ちしますをするロボット',
  'VR＋BOXの旅行体験トラック',
  '階段を登るのが楽になる靴や鞄',
  'VR/ARで道案内'])

In [160]:

new_idea_text = '外の景色を隠してくれる'
 #主成分分析　第二主成分だけ使う
component_num = 5
cdluster_id = 15
pca = PCA(n_components=component_num) #インスタンス生成
ideas_matrix_pca = pca.fit_transform(ideas_df2) #各主成分の値をベクトル化して返す　
ideas_df_pca = pd.DataFrame(ideas_matrix_pca, index=idea_texts)# データフレームに格納
ideas_df_pca.columns = ["PC{}".format(i+1) for i in range(component_num)]#name colums:PC1~2

#k_meansでクラスタリング　第二主成分でクラスタリング
kmeans = KMeans(n_clusters=cdluster_id, random_state=0).fit(ideas_df_pca)
ideas_df_pca['cluster_id'] = kmeans.labels_


#新しいアイデアがどのクラスターに分類されるか？
new_vectors = vectorizer.vectorize([new_idea_text], "REDUCE_MEAN")
new_idea_vec_df = pd.DataFrame(new_vectors, index=[new_idea_text])
new_idea_pca = pca.transform(new_idea_vec_df)
new_idea_pca_df = pd.DataFrame(new_idea_pca)
new_idea_pca_df.columns = ["PC{}".format(i+1) for i in range(component_num)]#name colums:PC1~
cluster_new_idea = kmeans.predict(new_idea_pca_df)


/Users/wyuki/opt/anaconda3/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:1767: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [161]:
ideas_df_pca

,PC1,PC2,PC3,PC4,PC5,cluster_id
ガンダムみたいな乗り物,-0.481744,-0.707274,0.723515,0.541875,-0.569628,0
美容院,2.981252,0.586385,0.720931,-0.448871,-0.726207,11
登り棒,1.486443,-0.651506,0.374809,-0.832757,-1.201869,1
ソーシャルディスタンスを保てる,-1.816110,-2.488872,-0.997520,-0.510697,0.285580,12
乗った人の犯罪係数がわかる,-1.679867,-0.901124,-0.757021,0.563453,-0.981749,0
...,...,...,...,...,...,...
VR＋BOXの旅行体験トラック,-0.169903,-0.776693,-0.640703,0.701631,-0.001805,0
階段を登るのが楽になる靴や鞄,-1.497288,-0.249002,-0.729452,-0.404006,-0.920749,0
回転寿司が家を通る,-0.700540,0.772829,-0.142955,-0.683901,-0.421037,9
移動式スーパーマーケット,2.485724,3.328985,-0.642567,-1.768397,0.938015,7


In [162]:
#どのクラスターに何が含まれているかの確認
cluster_dic = {}
for cluster_id in range(cdluster_id):
    cluster_contents = []
    for row_id in range(len(ideas_df_pca.values)):
        idea_cluster = ideas_df_pca.iat[row_id, 5]
        if cluster_id == idea_cluster:
            cluster_contents.append(idea_texts[row_id])
    cluster_dic[cluster_id] = cluster_contents
cluster_dic

{0: ['ガンダムみたいな乗り物',
  '乗った人の犯罪係数がわかる',
  'いるべき車線がARで表示される',
  '方向音痴でも運転できる',
  'ドラえもんみたいに３mm浮いている',
  '乗れば空間充電される',
  '絶対にドライブデートが成功する',
  '時間ぴったりの動画が流れる',
  '乗った車が自分のものになる',
  '実際より速く感じる車',
  'VR＋BOXの旅行体験トラック',
  '階段を登るのが楽になる靴や鞄',
  'VR/ARで道案内'],
 1: ['登り棒',
  '道路でスキー',
  '渋滞の解消',
  '窓に電子広告',
  '信号機や交差点のいらない自動車',
  '学校まで続く滑り台',
  '地上にない道路',
  'VRで理想の道路'],
 2: ['ベッド', 'イス', '遊牧民', 'タイムマシン', '人力', 'UFOキャッチャー', 'カラオケ', '馬', 'ラウンドアバウト'],
 3: ['家から遠隔操作できる車',
  'ドローンで運ばれる',
  '夜は荷物うを勝手に運ぶ車',
  'エンジンからオーダーメイドの車',
  'たくさんの車でグリッドコンピューティング',
  '乗ってる人全員で運転',
  '車ごと電車に乗る',
  '車に乗ったままライブ',
  '車の中でカラオケ',
  '車の中でVRオフィス'],
 4: ['水陸空両用車',
  '無人デリバリー専用車',
  'ブレスト車',
  '消臭車',
  '野生のタクシー',
  '道路を舗装する車',
  '道路を掃除する車',
  '移動式ランドリーカー'],
 5: ['タイヤが球体',
  '道路がポイント切り替え',
  'フロントにいろいろ表示',
  '外側が液晶画面',
  '外の広告とAmazonが連動',
  '助手席がDJ',
  '窓ガラスが液晶ディスプレイ',
  '目的地がランダムなタクシー',
  '書類を電子化するポスト'],
 6: ['ついてくる影', 'ついてくる傘', 'どこでも止まれる電車'],
 7: ['動くドライブスルー',
  '動く道路',
  '車っぽい買い物袋',
  '街中ジェットコースター',
  '動く一軒家',
  '移動式ライブハウス',


## ここからメモ

In [8]:
#K_means　ユークリッド距離でクラスタリングしてるのはいいのか？
# 
kmeans = KMeans(n_clusters=5, random_state=0).fit(ideas_df)
ideas_df['cluster_id'] = kmeans.labels_
kmeans.predict(new_idea_df)

array([4], dtype=int32)

In [71]:
#主成分分析　index:アイデア　colums:主成分ベクトル　のデータフレーム作成
component_num = 5
pca = PCA(n_components=component_num)
ideas_matrix_pca = pca.fit_transform(ideas_df)
ideas_df_pca = pd.DataFrame(ideas_matrix_pca) 
ideas_df_pca.columns = ["PC{}".format(i+1) for i in range(component_num)]
ideas_df_pca['cluster_id'] = kmeans.labels_
ideas_df_pca.head(20)

,PC1,PC2,PC3,PC4,PC5,cluster_id
0,-1.834162,-0.459048,0.758886,0.465615,0.711567,2
1,-2.925509,1.131740,-0.665730,-0.403743,-0.413718,1
2,-2.475665,0.683428,-0.463482,-0.926292,-0.163287,1
3,1.390676,-2.013708,0.586179,1.783044,-0.158605,2
4,3.359108,-2.570103,-0.381915,0.211285,-0.423757,0
5,-1.766243,-1.418777,0.640318,-0.186449,-0.066721,2
6,-2.695544,1.221805,-0.732972,0.238624,-0.128720,1
7,-3.943939,2.532999,-0.749757,-0.000815,-0.378727,1
8,-2.895611,0.728126,0.407825,1.176255,0.268642,1
9,-0.196105,-1.883974,0.659739,-0.815762,-0.185275,2


In [10]:
#クラスターごとに主成分スコアの平均値を計算
ideas_pca_mean_df = ideas_df_pca.groupby('cluster_id').mean()
ideas_pca_mean_df

,PC1,PC2,PC3,PC4,PC5
cluster_id,,,,,
0,4.769010,-2.258573,2.558939,0.001200,0.008610
1,-3.072545,1.068961,0.318156,-0.002344,-0.078439
2,-0.354045,-1.412653,-0.821246,0.009928,0.151344
3,11.328708,3.132512,-0.213423,1.827704,1.511615
4,10.575899,1.663534,-1.263371,-1.571739,-1.629326


In [11]:
#PC２の主成分スコアの平均値が最も高いクラスター
ideas_pca_mean_df['PC2'].idxmax()

3

In [63]:
#主成分分析　index:アイデア　colums:主成分ベクトル　のデータフレーム作成
component_num = 5
pca = PCA(n_components=component_num)
ideas_matrix_pca = pca.fit_transform(ideas_df)
ideas_df_pca = pd.DataFrame(ideas_matrix_pca) 
ideas_df_pca.columns = ["PC{}".format(i+1) for i in range(component_num)]
ideas_df_pca['cluster_id'] = ideas_df['cluster_id'].values
ideas_df_pca

,PC1,PC2,PC3,PC4,PC5,cluster_id
0,-1.766000,-0.376684,-1.081716,0.462638,-0.407661,2
1,-2.965866,1.104132,0.605432,-0.543268,0.208879,1
2,-2.518842,0.657810,0.407669,-0.765741,0.647526,1
3,1.432975,-1.983215,-0.187067,1.627697,-0.988917,2
4,3.155165,-2.765711,1.921796,0.445953,0.244837,0
...,...,...,...,...,...,...
106,0.022249,-1.684343,-0.681340,0.339494,-0.487111,2
107,10.127832,1.537964,-0.366557,-1.797454,-2.160282,4
108,1.031917,-2.305253,-0.522620,-0.324524,0.406257,2
109,-2.325330,0.360060,0.055321,0.947075,-0.816304,1


In [90]:
#コサイン類似度
def cos_sim(v1, v2):
    return np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))

In [177]:

#1つのデータと，各クラスタ中心との最高類似度
def max_cent_sim(idea_data, target_index, centroid_indexs):
    similarity= []
    for centroid_index in centroid_indexs:
        similarity.append(cos_sim(idea_data.iloc[target_index], idea_data.iloc[centroid_index]))
    return max(similarity)

In [178]:
#kkz法によるクラスタ中心初期化
def find_initial_centroids(idea_df, k):
    index = list(range(len(ideas_df)))
    pairs = list(itertools.combinations(index, 2)) #データの組み合わせ6105個　時間かかる
    distances_dic = {indexs:cos_sim(idea_df.iloc[indexs[0]], idea_df.iloc[indexs[1]]) for indexs in pairs}
    ini_centroids_index = list(max(distances_dic, key=distances_dic.get))
    while len(ini_centroids_index) < k:
        #データごとの，各クラスタ中心との最高類似度　{データインデックス：　最高類似度}の辞書
        candidate_centroids = {target_index: max_cent_sim(idea_df, target_index, ini_centroids_index) for target_index in index if target_index not in ini_centroids_index}
        #最高類似度群の，最低類似度
        ini_centroids_index.append(min(candidate_centroids, key=candidate_centroids.get))
    return ini_centroids_index


In [139]:
#K-means　skleanのKmeansがコサイン類似度を距離として使えなさそうなので実装
def kmeans_cos_kkz(idea_df, k):
    index = list(range(len(idea_df)))
    
    #kkzによる初期クラスタ設定
    ini_centroids_index = find_initial_centroids(idea_df, k)
    centroids = [idea_df.iloc[index] for index in ini_centroids_index]
    
    pre_centroids = []
    while set(pre_centroids) != set(centroids):
        pre_centroids = centroids
        clusters = {centroid:[] for in centroids} #クラスタリングされたデータの辞書　{クラスタ中心:[データインデックスのリスト]}
        
        #ideaデータごとに，最も似ているクラスタに分類
        for　target_idea_index in index:
            cent_sim_dic = {}
            for centroid in centroids:
                sim = cos_sim(centroid, idea_df.iloc[target_idea_index])
                if round(sim, 10) != 1: #データとクラスタ中心が同じ場合を排除
                    cent_sim_dic[centroid] = sim #データと各クラスタ中心との類似度を保持
            clusters[max(cent_sim_dic, key=cent_sim_dic.get)] = target_idea_index #最も類似度の高いクラスタ中心をキーとして，indexを保存
        
        #各クラスタの重心を計算
        
    

In [189]:
cos_sim(ideas_df.iloc[3], ideas_df.iloc[10])

0.8731866905530596

In [183]:
ideas_df.iloc[0]

0              -0.678439
1              -0.037695
2               0.176766
3              -0.205148
4              -0.528620
                  ...   
764            -0.320730
765            -0.251870
766            -1.182096
767             0.190484
cluster_id_e    2.000000
Name: ガンダムみたいな乗り物, Length: 769, dtype: float64

In [76]:
index = list(range(len(ideas_df)))
centroids = [15, 42]
print(index)
[index.remove(c) for c in centroids]
print(index)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110]


In [ ]:
#前作ったもの

In [30]:
#全データつっこんでベクトル化したやつ　　保存用
#アイデアデータをBERTでベクトル化
vectors = vectorizer.vectorize(idea_texts, "REDUCE_MEAN")
#データフレームにいれる
ideas_df = pd.DataFrame(vectors, index=idea_texts)
ideas_df

,0,1,2,3,4,5,6,7,8,9,...,758,759,760,761,762,763,764,765,766,767
ガンダムみたいな乗り物,-0.564032,0.125460,0.290597,-0.142004,-0.485771,0.120005,-0.283703,-0.291506,0.043798,0.634663,...,-0.272563,0.078560,-0.236389,-0.048522,0.122126,-0.420348,-0.292917,-0.202604,-1.180861,0.262144
美容院,-0.573068,0.321896,0.120986,-0.145179,-0.282246,0.384947,-0.042611,-0.178583,0.027171,0.565928,...,-0.104718,0.107351,-0.132360,0.046144,-0.081895,-0.466713,-0.272575,-0.324122,-1.323319,0.141219
登り棒,-0.579377,0.405865,0.035093,-0.132074,-0.259099,0.190102,-0.109147,-0.204407,-0.067220,0.675051,...,-0.039364,0.228805,-0.108255,0.013861,-0.008626,-0.535727,-0.243319,-0.341249,-1.279388,0.120697
ソーシャルディスタンスを保てる,-0.056543,0.486242,0.025704,-0.085814,0.016284,-0.046788,-0.271251,-0.044203,0.319122,0.443550,...,-0.320670,0.105553,-0.146231,-0.058548,0.153158,-0.554334,-0.117725,-0.337645,-0.704658,0.285718
乗った人の犯罪係数がわかる,-0.323818,0.421837,0.115608,-0.341447,-0.276074,0.111587,-0.364390,-0.157841,0.123276,0.442815,...,-0.201161,-0.010786,0.045521,0.128644,-0.044046,-0.561812,0.107557,-0.570321,-0.259879,0.326136
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
VR＋BOXの旅行体験トラック,-0.471193,0.201763,-0.048822,-0.408934,-0.477877,0.022826,-0.030250,-0.145304,-0.169345,0.527150,...,-0.181793,0.129114,-0.015532,0.227496,0.071757,-0.578616,-0.420188,-0.217936,-0.764948,0.199017
階段を登るのが楽になる靴や鞄,-0.147847,-0.100502,-0.213684,-0.058233,-0.520064,0.051426,-0.049218,0.020794,-0.218314,-0.021853,...,-0.095379,0.008326,0.206203,0.103636,-0.040388,0.059366,-0.019810,-0.574022,0.075346,0.034326
回転寿司が家を通る,-0.294385,0.262505,0.232077,-0.312795,-0.199960,-0.186024,0.079747,-0.125565,0.121858,0.411492,...,-0.123509,0.029385,0.034136,-0.018251,0.044127,-0.576268,0.067588,-0.341484,-0.626641,0.204639
移動式スーパーマーケット,-0.572483,0.170701,0.282104,-0.122167,-0.439089,0.077927,-0.009855,-0.297955,-0.197106,0.604934,...,-0.258171,0.205824,-0.045972,0.052054,-0.105054,-0.669124,-0.250581,-0.169513,-1.103971,0.172342


In [76]:
def new_idea_is_hint(ideas_df, idea_texts, new_idea_text):
    #k_meansでクラスタリング
    kmeans = KMeans(n_clusters=5, random_state=0).fit(ideas_df)
    
    #新しいアイデアがどのクラスターに分類されるか？
    new_vectors = vectorizer.vectorize([new_idea_text], "CLS_TOKEN")
    new_idea_vec_df = pd.DataFrame(new_vectors, index=[new_idea_text])
    cluster_new_idea = kmeans.predict(new_idea_vec_df)[0]
    print('cluster_new_idea: ', cluster_new_idea)
    
    #主成分分析　第二主成分だけ残す
    component_num = 5
    pca = PCA(n_components=component_num) #インスタンス生成
    ideas_matrix_pca = pca.fit_transform(ideas_df) #各主成分の値をベクトル化して返す　
    ideas_df_pca = pd.DataFrame(ideas_matrix_pca)# データフレームに格納
    ideas_df_pca.columns = ["PC{}".format(i+1) for i in range(component_num)]#name colums:PC1~5
    ideas_df_pca['cluster_id'] = kmeans.labels_ #クラスターラベルの列を追加
    ideas_pca_mean_df = ideas_df_pca.groupby('cluster_id').mean()#クラスターごとに集約
    cluster_maybe_hint = ideas_pca_mean_df['PC2'].idxmax()#PC2の値が最も大きいクラスターidを取得
    print('cluster_maybe_hint: ', cluster_maybe_hint)
    
    return cluster_new_idea == cluster_maybe_hint